In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\Daniel'

In [3]:
os.chdir("E:\python")

In [4]:
os.getcwd()

'E:\\python'

In [5]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt

In [6]:
df=pd.read_csv('x_train.csv') # read the train data from the file

In [7]:
df.isnull().values.sum() #count the numbers of missing value in the data

0

In [8]:
data=df.iloc[:,1:6] # get rid of the GeneId column

In [9]:
mydata=data.values   #convert the data frame to numpy array
mydata=mydata.flatten() #convert the n-dimension array to one-dimension array

In [10]:
mydata=np.reshape(mydata,(15485,500)) #reshape the array so that there are 500 columns each row

In [11]:
mydata

array([[2, 1, 4, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 6, 3, ..., 4, 3, 0],
       ..., 
       [0, 2, 1, ..., 2, 0, 1],
       [0, 0, 0, ..., 2, 0, 0],
       [2, 0, 0, ..., 0, 2, 1]], dtype=int64)

In [12]:
from sklearn import preprocessing

In [13]:
robust_scaler=preprocessing.RobustScaler()

In [14]:
x_train=robust_scaler.fit_transform(mydata)

In [15]:
df=pd.read_csv('x_test.csv')

In [16]:
df=df.iloc[:,1:6]

In [17]:
df=df.values

In [18]:
df=df.reshape(3871,500)

In [19]:
df


array([[ 2,  3,  4, ...,  6,  2,  3],
       [ 1,  4,  2, ...,  3,  3,  1],
       [ 0,  6,  4, ...,  7,  1,  2],
       ..., 
       [ 2,  3, 13, ...,  3,  0,  0],
       [ 1,  2,  1, ...,  1,  0,  2],
       [ 1,  4,  4, ...,  0,  0,  0]], dtype=int64)

In [20]:
x_test=robust_scaler.transform(df)

In [21]:
df=pd.read_csv('y_train.csv')

In [22]:
df=df.iloc[:,1:2]

In [23]:
y_train=df.values

In [24]:
y_train=y_train.flatten()

In [25]:
y_train

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
from sklearn.model_selection import cross_val_score

In [28]:
from sklearn import svm

In [29]:
clf = RandomForestClassifier(random_state=0)

In [30]:
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc') # using preprocessed data

In [31]:
scores

array([ 0.90143336,  0.89711852,  0.89714519,  0.89330529,  0.88141193])

In [32]:
scores = cross_val_score(clf, mydata, y_train, cv=5, scoring='roc_auc') # using raw data

In [33]:
scores

array([ 0.90148317,  0.89720479,  0.89711036,  0.89345302,  0.88132283])

In [34]:
from sklearn import linear_model

In [35]:
logreg = linear_model.LogisticRegression()

In [36]:
scores = cross_val_score(logreg,x_train,y_train,cv=10)

In [37]:
scores

array([ 0.87088444,  0.84118786,  0.85087153,  0.8431246 ,  0.85022595,
        0.8431246 ,  0.83925113,  0.85529716,  0.81383323,  0.83193277])

In [38]:
from sklearn.neural_network import MLPClassifier

In [39]:
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 2), random_state=1)

In [40]:
scores = cross_val_score(clf,x_train,y_train,cv=5)

In [41]:
scores

array([ 0.85248547,  0.84796643,  0.84662577,  0.84011628,  0.8123385 ])

In [42]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

In [43]:
cv = StratifiedKFold(n_splits=6)
classifier1 = linear_model.LogisticRegression()
classifier2 = svm.SVC(kernel='linear', probability=True, random_state=1)
classifier3 = RandomForestClassifier(random_state=1)
classifier4 = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(30, 2), random_state=1)

In [44]:
aucs = []

In [45]:
for train, test in cv.split(x_train, y_train):
    probas_ = classifier4.fit(x_train[train], y_train[train]).predict_proba(x_train[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_train[test], probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

In [46]:
aucs

[0.90172837015367024,
 0.89252178476555566,
 0.89572675130780666,
 0.86954209959985207,
 0.89231672695306308,
 0.86299888948714498]